In [ ]:
# Import libraries and loading the csv file
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
import missingno as msno
import plotly.express as px
import re
import sweetviz as sv
from kmodes.kprototypes import KPrototypes
import scipy.stats as stats
from scipy.stats import chi2
vehicles_data_initial = pd.read_csv(r"C:\Users\91886\OneDrive\QMUL Masterclass\vehicles.csv")

In [ ]:
vehicles_data =  vehicles_data_initial.copy()
sample = vehicles_data_initial.copy()

In [ ]:
vehicles_data[vehicles_data["price"] < 500]

In [ ]:
vehicles_data.drop(vehicles_data[vehicles_data['price'] < 500].index, inplace = True)

In [ ]:
vehicles_data.drop(['url', 'region_url','image_url','county','VIN','size'], axis=1,inplace = True)

In [ ]:
vehicles_data.shape

In [ ]:
msno.matrix(vehicles_data, color = (0.5, 0.5, 0.5))

In [ ]:
percent_missing = vehicles_data.isnull().sum() * 100 / len(vehicles_data)
missing_value_df = pd.DataFrame({'column_name': vehicles_data.columns,
                                 'percent_missing': percent_missing})

In [ ]:
missing_value_df

In [ ]:
vehicles_data.drop(vehicles_data[vehicles_data['year'] < 1990].index, inplace = True)

In [ ]:
vehicles_data.dropna(axis = 0,thresh=17,inplace = True)

In [ ]:
vehicles_data.dropna(subset=['manufacturer', 'model','odometer','title_status','fuel',
                               'transmission','lat','long','year','description'], inplace = True)

In [ ]:
msno.matrix(vehicles_data, color = (0.5, 0.5, 0.5))

In [ ]:
percent_missing = vehicles_data.isnull().sum() * 100 / len(vehicles_data)
missing_value_df = pd.DataFrame({'column_name': vehicles_data.columns,
                                 'percent_missing': percent_missing})
missing_value_df

In [ ]:
vehicles_data.shape

In [ ]:
# To find records with fuel as electric and cylinders as NULL
condition1 = pd.isna(vehicles_data_initial["cylinders"])
condition2 = vehicles_data["fuel"] == 'electric'

In [ ]:
# Use the & operator to combine the conditions
filtered_df = vehicles_data[condition1 & condition2]

In [ ]:
# imputing electric car cylinders to 0
vehicles_data['cylinders'] = np.where(pd.isna(vehicles_data["cylinders"]) & (vehicles_data["fuel"] == 'electric')
                                      ,0,vehicles_data['cylinders'])

In [ ]:
# Data correction for fuel type as electric and number of cylinders as non zero
vehicles_data[vehicles_data["fuel"] == 'electric']["cylinders"].value_counts()
vehicles_data['cylinders'] = np.where((vehicles_data['fuel'] == 'electric') & (vehicles_data['cylinders'] != 0), 
                                         0, vehicles_data['cylinders'])

In [ ]:
# Dropping Null values from Type

vehicles_data.dropna(subset=['type'], inplace = True)

In [ ]:
vehicles_data.shape

In [ ]:
percent_missing = vehicles_data.isnull().sum() * 100 / len(vehicles_data)
missing_value_df = pd.DataFrame({'column_name': vehicles_data.columns,
                                 'percent_missing': percent_missing})

In [ ]:
missing_value_df

msno.matrix(vehicles_data, color = (0.5, 0.5, 0.5))

# Handing outliers

In [ ]:
df = vehicles_data[vehicles_data['year'] == 2007]["price"]
print(df.max())

In [ ]:
df = vehicles_data[vehicles_data['year'] == 1999]["price"]
print(df.max())

In [ ]:
filtered_df = vehicles_data[vehicles_data["price"] == 3736928711]

In [ ]:
vehicles_data.drop(vehicles_data[vehicles_data['price'] == 3736928711].index, inplace = True)

In [ ]:
vehicles_data.drop(vehicles_data[vehicles_data['price'] == 1111111111].index, inplace = True)

In [ ]:
df = vehicles_data[vehicles_data['year'] == 2015]["price"]
print(df.max())

In [ ]:
# removing Outlier prices
vehicles_data.drop(vehicles_data[vehicles_data['price'] == 3736928711].index, inplace = True)
vehicles_data.drop(vehicles_data[vehicles_data['price'] == 1111111111].index, inplace = True)
vehicles_data.drop(vehicles_data[vehicles_data['price'] == 123456789].index, inplace = True)
vehicles_data.drop(vehicles_data[vehicles_data['price'] == 17000000].index, inplace = True)
vehicles_data.drop(vehicles_data[vehicles_data['price'] == 6995495].index, inplace = True)
vehicles_data.drop(vehicles_data[vehicles_data['price'] == 2000000].index, inplace = True)
vehicles_data.drop(vehicles_data[vehicles_data['price'] == 1234567].index, inplace = True)
vehicles_data.drop(vehicles_data[vehicles_data['price'] == 990000].index, inplace = True)
vehicles_data.drop(vehicles_data[vehicles_data['price'] == 135008900].index, inplace = True)

In [ ]:
fig = px.scatter(vehicles_data, x = 'odometer', y = 'price', color = 'fuel')
fig.show()

In [ ]:
# removing odometer > 5000000
vehicles_data.drop(vehicles_data[vehicles_data['odometer'] >= 5000000].index,inplace = True)

In [ ]:
fig = px.scatter(vehicles_data, x = 'odometer', y = 'price', color = 'fuel')
fig.show()

In [ ]:
# copy the data for analysis to find correlation for non missing cylinder values
vehicles_data_cyldrop = vehicles_data.copy()

In [ ]:
vehicles_data_cyldrop.dropna(subset=['cylinders','drive','condition','paint_color'], inplace = True)

In [ ]:
fig = px.scatter(vehicles_data_cyldrop, x = 'cylinders', y = 'price', color = 'cylinders')
fig.show()

In [ ]:
# replace the huge name of model with a short value for a better graph
vehicles_data_cyldrop['model'] = np.where(vehicles_data_cyldrop['model'].str.contains('impreza sedan premium')
                                          , 'impreza', vehicles_data_cyldrop['model'])

In [ ]:
fig = px.scatter(vehicles_data_cyldrop, x = 'paint_color', y = 'price', color = 'manufacturer')
fig.show()

In [ ]:
filtered_df = vehicles_data[vehicles_data['paint_color'].isnull()]

# Null values analysis on basis of Description

In [ ]:
def extract_color(expression):
    pattern = r"Color: ([A-Za-z]+)"
    color = re.search(pattern, expression)
    if color:
        return color.group(1)
    return None

In [ ]:
filtered_df["color"] = filtered_df["description"].apply(extract_color)

In [ ]:
filtered_df["color"].value_counts()

In [ ]:
vehicles_data.isna().sum()

In [ ]:
vehicles_data_paint = vehicles_data.copy()

In [ ]:
def extract_color(expression):
    pattern = r"Color: ([A-Za-z]+)"
    color = re.search(pattern, expression)
    if color:
        return color.group(1)
    return None

In [ ]:
vehicles_data_paint["paint_color"] = vehicles_data_paint["paint_color"].where(~vehicles_data_paint["paint_color"].isnull(), 
                                                                              vehicles_data_paint["description"].apply(extract_color))

In [ ]:
vehicles_data_paint["paint_color"].unique()

# From Drive column

In [ ]:
def extract_drive_type(expression):
    pattern = r"Drive: ([A-Za-z0-9]+)"
    drive_type = re.search(pattern, expression)
    if drive_type:
        return drive_type.group(1)
    return None

In [ ]:
vehicles_data_paint["drive"] = vehicles_data_paint["drive"].where(~vehicles_data_paint["drive"].isnull(), 
                                                                  vehicles_data_paint["description"].apply(extract_drive_type))

In [ ]:
vehicles_data_paint["drive"].value_counts()

In [ ]:
vehicles_data_paint.isna().sum()

In [ ]:
vehicles_data_cyldrop

fig = px.scatter(vehicles_data_cyldrop, x = 'drive', y = 'price', color = 'cylinders')
fig.show()

In [ ]:
fig = px.scatter(vehicles_data_cyldrop, x = 'type', y = 'price', color = 'cylinders')
fig.show()

In [ ]:
fig = px.scatter(vehicles_data_cyldrop, x = 'paint_color', y = 'price', color = 'model')
fig.show()

In [ ]:
# Import libraries and loading the csv file
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
import missingno as msno
import plotly.express as px
import sweetviz as sv
from kmodes.kprototypes import KPrototypes
import scipy.stats as stats
from scipy.stats import chi2
vehicles_data_initial = pd.read_csv(r"C:\Users\91886\OneDrive\QMUL Masterclass\vehicles.csv")

In [ ]:
vehicles_data =  vehicles_data_initial.copy()
sample = vehicles_data_initial.copy()

In [ ]:
vehicles_data[vehicles_data["price"] < 500]

In [ ]:
vehicles_data.drop(vehicles_data[vehicles_data['price'] < 500].index, inplace = True)

In [ ]:
vehicles_data.drop(['url', 'region_url','image_url','county','VIN','size'], axis=1,inplace = True)

In [ ]:
vehicles_data.shape

In [ ]:
msno.matrix(vehicles_data, color = (0.5, 0.5, 0.5))

In [ ]:
percent_missing = vehicles_data.isnull().sum() * 100 / len(vehicles_data)
missing_value_df = pd.DataFrame({'column_name': vehicles_data.columns,
                                 'percent_missing': percent_missing})

In [ ]:
missing_value_df

In [ ]:
vehicles_data.drop(vehicles_data[vehicles_data['year'] < 1990].index, inplace = True)

In [ ]:
vehicles_data.dropna(axis = 0,thresh=17,inplace = True)

In [ ]:
vehicles_data.dropna(subset=['manufacturer', 'model','odometer','title_status','fuel',
                               'transmission','lat','long','year','description'], inplace = True)

In [ ]:
msno.matrix(vehicles_data, color = (0.5, 0.5, 0.5))

In [ ]:
percent_missing = vehicles_data.isnull().sum() * 100 / len(vehicles_data)
missing_value_df = pd.DataFrame({'column_name': vehicles_data.columns,
                                 'percent_missing': percent_missing})
missing_value_df

In [ ]:
vehicles_data.shape

In [ ]:
# To find records with fuel as electric and cylinders as NULL
condition1 = pd.isna(vehicles_data_initial["cylinders"])
condition2 = vehicles_data["fuel"] == 'electric'

In [ ]:
# Use the & operator to combine the conditions
filtered_df = vehicles_data[condition1 & condition2]

In [ ]:
# imputing electric car cylinders to 0
vehicles_data['cylinders'] = np.where(pd.isna(vehicles_data["cylinders"]) & (vehicles_data["fuel"] == 'electric')
                                      ,0,vehicles_data['cylinders'])

In [ ]:
# Data correction for fuel type as electric and number of cylinders as non zero
vehicles_data[vehicles_data["fuel"] == 'electric']["cylinders"].value_counts()
vehicles_data['cylinders'] = np.where((vehicles_data['fuel'] == 'electric') & (vehicles_data['cylinders'] != 0), 
                                         0, vehicles_data['cylinders'])

In [ ]:
# Dropping Null values from Type

vehicles_data.dropna(subset=['type'], inplace = True)

In [ ]:
vehicles_data.shape

In [ ]:
percent_missing = vehicles_data.isnull().sum() * 100 / len(vehicles_data)
missing_value_df = pd.DataFrame({'column_name': vehicles_data.columns,
                                 'percent_missing': percent_missing})

In [ ]:
missing_value_df

In [ ]:
msno.matrix(vehicles_data, color = (0.5, 0.5, 0.5))

# Handing outliers

In [ ]:
df = vehicles_data[vehicles_data['year'] == 2007]["price"]
print(df.max())

In [ ]:
df = vehicles_data[vehicles_data['year'] == 1999]["price"]
print(df.max())

In [ ]:
filtered_df = vehicles_data[vehicles_data["price"] == 3736928711]

In [ ]:
vehicles_data.drop(vehicles_data[vehicles_data['price'] == 3736928711].index, inplace = True)

In [ ]:
vehicles_data.drop(vehicles_data[vehicles_data['price'] == 1111111111].index, inplace = True)

In [ ]:
df = vehicles_data[vehicles_data['year'] == 2015]["price"]
print(df.max())

In [ ]:
# removing Outlier prices
vehicles_data.drop(vehicles_data[vehicles_data['price'] == 3736928711].index, inplace = True)
vehicles_data.drop(vehicles_data[vehicles_data['price'] == 1111111111].index, inplace = True)
vehicles_data.drop(vehicles_data[vehicles_data['price'] == 123456789].index, inplace = True)
vehicles_data.drop(vehicles_data[vehicles_data['price'] == 17000000].index, inplace = True)
vehicles_data.drop(vehicles_data[vehicles_data['price'] == 6995495].index, inplace = True)
vehicles_data.drop(vehicles_data[vehicles_data['price'] == 2000000].index, inplace = True)
vehicles_data.drop(vehicles_data[vehicles_data['price'] == 1234567].index, inplace = True)
vehicles_data.drop(vehicles_data[vehicles_data['price'] == 990000].index, inplace = True)
vehicles_data.drop(vehicles_data[vehicles_data['price'] == 135008900].index, inplace = True)

In [ ]:
fig = px.scatter(vehicles_data, x = 'year', y = 'price', color = 'fuel')
fig.show()

In [ ]:
fig = px.scatter(vehicles_data, x = 'odometer', y = 'price', color = 'fuel')
fig.show()

In [ ]:
# removing odometer > 5000000
vehicles_data.drop(vehicles_data[vehicles_data['odometer'] >= 5000000].index,inplace = True)

In [ ]:
fig = px.scatter(vehicles_data, x = 'odometer', y = 'price', color = 'fuel')
fig.show()

In [ ]:
# copy the data for analysis to find correlation for non missing cylinder values
vehicles_data_cyldrop = vehicles_data.copy()

In [ ]:

vehicles_data_cyldrop.dropna(subset=['cylinders','drive','condition','paint_color'], inplace = True)

In [ ]:
fig = px.scatter(vehicles_data_cyldrop, x = 'cylinders', y = 'price', color = 'cylinders')
fig.show()

In [ ]:
# replace the huge name of model with a short value for a better graph
vehicles_data_cyldrop['model'] = np.where(vehicles_data_cyldrop['model'].str.contains('impreza sedan premium')
                                          , 'impreza', vehicles_data_cyldrop['model'])

In [ ]:
fig = px.scatter(vehicles_data_cyldrop, x = 'paint_color', y = 'price', color = 'manufacturer')
fig.show()

In [ ]:
filtered_df = vehicles_data[vehicles_data['paint_color'].isnull()]

# Null values analysis on basis of Description

In [ ]:
def extract_color(expression):
    pattern = r"Color: ([A-Za-z]+)"
    color = re.search(pattern, expression)
    if color:
        return color.group(1)
    return None

In [ ]:
filtered_df["color"] = filtered_df["description"].apply(extract_color)

In [ ]:
filtered_df["color"].value_counts()

In [ ]:
vehicles_data.isna().sum()

In [ ]:
vehicles_data_paint = vehicles_data.copy()

In [ ]:
def extract_color(expression):
    pattern = r"Color: ([A-Za-z]+)"
    color = re.search(pattern, expression)
    if color:
        return color.group(1)
    return None

In [ ]:
vehicles_data_paint["paint_color"] = vehicles_data_paint["paint_color"].where(~vehicles_data_paint["paint_color"].isnull(), 
                                                                              vehicles_data_paint["description"].apply(extract_color))

In [ ]:
vehicles_data_paint["paint_color"].unique()

# From Drive column

In [ ]:
def extract_drive_type(expression):
    pattern = r"Drive: ([A-Za-z0-9]+)"
    drive_type = re.search(pattern, expression)
    if drive_type:
        return drive_type.group(1)
    return None

In [ ]:
vehicles_data_paint["drive"] = vehicles_data_paint["drive"].where(~vehicles_data_paint["drive"].isnull(), 
                                                                  vehicles_data_paint["description"].apply(extract_drive_type))

In [ ]:
vehicles_data_paint["drive"].value_counts()

In [ ]:
vehicles_data_paint.isna().sum()

In [ ]:
vehicles_data_cyldrop

fig = px.scatter(vehicles_data_cyldrop, x = 'drive', y = 'price', color = 'cylinders')
fig.show()

In [ ]:
fig = px.scatter(vehicles_data_cyldrop, x = 'type', y = 'price', color = 'cylinders')
fig.show()

In [ ]:
fig = px.scatter(vehicles_data_cyldrop, x = 'paint_color', y = 'price', color = 'model')
fig.show()